In [1]:
%pylab inline
from scipy import interpolate
from scipy import optimize

Populating the interactive namespace from numpy and matplotlib


In [2]:
#U = 0 # coulob repulsion
#ne = 1.6 # number of electrons per unit cell/ occupation number
# The occupation number independent part of the Hamiltonina
nk = 200
kran = linspace(-pi,pi,nk,endpoint=False)
kx,ky = meshgrid(kran,kran)
kx = kx.flatten()
ky = ky.flatten()
gamma = -(1+exp(-1j*2*kx)+exp(-1j*(kx-ky))+exp(-1j*(kx+ky)))
H0k = zeros((len(gamma),2,2),dtype=complex)
H0k[:,0,1] = gamma
H0k[:,1,0] = conj(gamma)


In [3]:
# this generates eigen values and eigen vectors and returns them 
def make_spectr(U,n1,n2):
    Hk = H0k.copy()
    Hk[:,0,0] = U*n1
    Hk[:,1,1] = U*n2
    return eigh(Hk)

def FD(Energy,kbt):
    return 1/(1+exp(Energy/kbt))


In [5]:
# a function that would find the new occupation number for each site, it needs to be run in a self consistency loop, this vergion can be used with fixed point f'n     
def self_consistency2(ns,U,ne,kbt):  # (U,ns,ne)
        n1_do,n2_do,n1_up,n2_up = ns  

        vals_up,vects_up = make_spectr(U,n1_do,n2_do)
        vals_do,vects_do = make_spectr(U,n1_up,n2_up)
        num_of_k_points = len(vals_up)

        
        #########################################################################
        # get new spectrum and wavefunctions
        # vals_up[ik,i] the i-th eigenvalue of up electrons at the ik-th k point
        # vecs_up[ik,site,i] the i-th wavefunction of up electrons at the ik-th k point evaluated at site "site".
 
        
        rho,eran = histogram(array([vals_up,vals_do]).flatten(),1000,density=True)
        rho = convolve(rho,hamming(10),mode='same')/5 # some additional smoothing
        NE = cumsum(rho)*diff(eran)[0]
        mu = interpolate.interp1d(NE*4,eran[:-1],fill_value="extrapolate")
        

        
        
        n1_do_new = (sum(abs(vects_do[:, 0, 0])**2*FD(vals_do[:,0]-mu(ne),kbt))+
                    sum(abs(vects_do[:, 0, 1])**2*FD(vals_do[:,1]-mu(ne),kbt)))/num_of_k_points
                                                                                                                            #i should find fermi function of energy - mu
        n2_do_new = (sum(abs(vects_do[:, 1, 0])**2*FD(vals_do[:,0]-mu(ne),kbt))+
                    sum(abs(vects_do[:, 1, 1])**2*FD(vals_do[:,1]-mu(ne),kbt)))/num_of_k_points
        
        n1_up_new = (sum(abs(vects_up[:, 0, 0])**2*FD(vals_up[:,0]-mu(ne),kbt))+
                    sum(abs(vects_up[:, 0, 1])**2*FD(vals_up[:,1]-mu(ne),kbt)))/num_of_k_points
        
        n2_up_new =  (sum(abs(vects_up[:, 1, 0])**2*FD(vals_up[:,0]-mu(ne),kbt))+
                    sum(abs(vects_up[:, 1, 1])**2*FD(vals_up[:,1]-mu(ne),kbt)))/num_of_k_points
        
        if ((n1_up > n1_do) and (n1_up_new < n1_do_new)) or ((n1_do > n1_up) and (n1_do_new < n1_up_new)) :
            n1_up_new,n1_do_new = n1_do_new,n1_do_new
        if ((n2_up > n2_do) and (n2_up_new < n2_do_new)) or ((n2_do > n2_up) and (n2_do_new < n2_up_new)):
            n2_up_new,n2_do_new = n2_do_new,n2_do_new
            
        
        ns_new = array([n1_do_new ,n2_do_new, n1_up_new, n2_up_new])
        ns_new = (ns_new * ne)/ sum(ns_new)

        # this is a comment
       
        return ns_new - ns

In [6]:
# kbt =.01
# # U = 1/.2
# ne = 1.6
# delta=0.1
# n1_up = (ne/4) + delta #occupation on site 1
# n1_do = (ne/4) - delta
# n2_up = (ne/4) - delta
# n2_do = (ne/4) + delta

# # dd = optimize.fixed_point(self_consistency2,[n1_do,n2_do, n1_up,n2_up ], args=(U,ne))   # another method using fixed point function to find the self consistant solution
# # print(dd)

# # dd = self_consistency2(n1_do,n2_do,n1_up,n2_up,U,ne)
# # print(dd)

In [17]:
kbt =.01
# U = 1/.2
ne = 1.6
delta=.4
n1_up = (ne/4) + delta #occupation on site 1
n1_do = (ne/4) - delta
n2_up = (ne/4) - delta
n2_do = (ne/4) + delta
nvals_anti = []
for U in (1/arange(.1,.24,.01)):
    nvals_anti.append(optimize.root(self_consistency2,[n1_do,n2_do, n1_up,n2_up ], args=(U,ne,kbt),method='broyden2',options={'disp':True}))

/home/assem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


0:  |F(x)| = 0.0258936; step 0.0872919
1:  |F(x)| = 0.0005518; step 1
2:  |F(x)| = 7.82493e-05; step 1
3:  |F(x)| = 7.02902e-06; step 1
4:  |F(x)| = 1.23122e-06; step 1
0:  |F(x)| = 0.0203649; step 0.14434
1:  |F(x)| = 0.00102352; step 1
2:  |F(x)| = 0.000104119; step 1
3:  |F(x)| = 6.56268e-05; step 1
4:  |F(x)| = 5.92056e-05; step 0.201132
5:  |F(x)| = 0.000224411; step 1
6:  |F(x)| = 1.36947e-05; step 1
7:  |F(x)| = 2.73014e-06; step 0.427363
0:  |F(x)| = 0.0095164; step 0.231037
1:  |F(x)| = 0.000556122; step 1
2:  |F(x)| = 1.71043e-05; step 1
3:  |F(x)| = 2.96412e-06; step 1
0:  |F(x)| = 0.00778027; step 0.35986
1:  |F(x)| = 0.000731354; step 1
2:  |F(x)| = 0.000147158; step 1
3:  |F(x)| = 1.94951e-05; step 1
4:  |F(x)| = 5.85604e-08; step 1
0:  |F(x)| = 0.0688952; step 1
1:  |F(x)| = 0.0256985; step 1
2:  |F(x)| = 0.0017943; step 0.493394
3:  |F(x)| = 0.00017713; step 1
4:  |F(x)| = 1.22978e-05; step 1
5:  |F(x)| = 6.22213e-06; step 1
6:  |F(x)| = 5.51267e-08; step 1
0:  |F(x)| =

In [18]:
kbt =.01
# U = 1/.2
ne = 1.6
delta=0.1
n1_up = (ne/4) + delta #occupation on site 1
n1_do = (ne/4) - delta
n2_up = (ne/4) + delta
n2_do = (ne/4) - delta
nvals_fero = []
for U in (1/arange(.1,.24,.01)):
    nvals_fero.append(optimize.root(self_consistency2,[n1_do,n2_do, n1_up,n2_up ], args=(U,ne,kbt),method='broyden2',options={'disp':True}))

0:  |F(x)| = 0.05; step 1


/home/assem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


1:  |F(x)| = 0.0463873; step 0.00820481
2:  |F(x)| = 4.10783e-15; step 1
0:  |F(x)| = 0.05; step 1
1:  |F(x)| = 0.0483094; step 0.00202122
2:  |F(x)| = 2.07612e-14; step 1
0:  |F(x)| = 0.0449178; step 1
1:  |F(x)| = 0.0447792; step 0.000605049
2:  |F(x)| = 0.042987; step 0.0221154
3:  |F(x)| = 0.0293494; step 0.100997
4:  |F(x)| = 0.000871345; step 1
5:  |F(x)| = 3.04978e-13; step 1
0:  |F(x)| = 0.0238405; step 1
1:  |F(x)| = 0.0235158; step 0.00421887
2:  |F(x)| = 0.0201611; step 0.131315
3:  |F(x)| = 0.0172434; step 0.138063
4:  |F(x)| = 0.0151277; step 0.121825
5:  |F(x)| = 0.0131383; step 0.0961313
6:  |F(x)| = 0.00634745; step 0.210871
7:  |F(x)| = 0.000291046; step 1
8:  |F(x)| = 7.77899e-10; step 1
0:  |F(x)| = 0.00515017; step 1
1:  |F(x)| = 0.00284526; step 0.174117
2:  |F(x)| = 6.93424e-05; step 1
3:  |F(x)| = 2.20169e-05; step 1
4:  |F(x)| = 9.34634e-06; step 1
5:  |F(x)| = 2.49121e-06; step 1
0:  |F(x)| = 0.0116359; step 1
1:  |F(x)| = 0.0101263; step 1
2:  |F(x)| = 0.00076